# template

In [ ]:
from collections import OrderedDict, defaultdict
from typing import Callable, Tuple, Dict, List

import numpy as np
from tqdm.auto import tqdm


def distance(pointA: np.ndarray, documents: np.ndarray) -> np.ndarray:
    # допишите ваш код здесь 
    pass


def create_sw_graph(
        data: np.ndarray,
        num_candidates_for_choice_long: int = 10,
        num_edges_long: int = 5,
        num_candidates_for_choice_short: int = 10,
        num_edges_short: int = 5,
        use_sampling: bool = False,
        sampling_share: float = 0.05,
        dist_f: Callable = distance
    ) -> Dict[int, List[int]]:
    # допишите ваш код здесь 
    pass

def nsw(query_point: np.ndarray, all_documents: np.ndarray, 
        graph_edges: Dict[int, List[int]],
        search_k: int = 10, num_start_points: int = 5,
        dist_f: Callable = distance) -> np.ndarray:
    # допишите ваш код здесь 
    pass



# code to send

In [1]:
from collections import OrderedDict, defaultdict
from typing import Callable, Tuple, Dict, List

import numpy as np
from tqdm.auto import tqdm


def distance(pointA: np.ndarray, documents: np.ndarray) -> np.ndarray:
    # допишите ваш код здесь 
    return np.sqrt(np.sum((documents - pointA) ** 2, axis=1)).reshape(-1,1)


def create_sw_graph(
        data: np.ndarray,
        num_candidates_for_choice_long: int = 10,
        num_edges_long: int = 5,
        num_candidates_for_choice_short: int = 10,
        num_edges_short: int = 5,
        use_sampling: bool = False,
        sampling_share: float = 0.05,
        dist_f: Callable = distance
    ) -> Dict[int, List[int]]:
    # допишите ваш код здесь 
    dict_points = dict()
    
    for i in range(np.shape(data)[0]):
        # random sampling for large datasets
        if use_sampling:
            N_points = int(sampling_share * np.shape(data)[0])
            N_points_idx = np.random.choice(np.shape(data)[0], N_points, replace=False)
            # distance from current point
            dist_i = distance(data[i, ], data[N_points_idx, ])
        else:
            dist_i = distance(data[i, ], data)
        
        # get indices of sorted values
        dist_i_argsort = np.argsort(dist_i, axis=0)
        
        # choose long edges (points)
        num_edges_long = np.min((num_candidates_for_choice_long, num_edges_long))
        long_rand_idx = np.random.choice(num_candidates_for_choice_long, num_edges_long,\
                                         replace=False)
        long_idx = dist_i_argsort[-num_candidates_for_choice_long:][long_rand_idx]
        
        
        # choose short edges (points)
        num_edges_short = np.min((num_candidates_for_choice_short, num_edges_short))
        short_rand_idx = np.random.choice(num_candidates_for_choice_short, num_edges_short,\
                                     replace=False)
        short_idx = dist_i_argsort[1:num_candidates_for_choice_short+1][short_rand_idx]
        
        # collect points' indices to dict
#         dict_points[i] = np.unique(np.concatenate(\
#             (long_idx, short_idx), axis=0).reshape(1,-1)[0]).tolist()
        dict_points[i] = np.concatenate(\
            (long_idx, short_idx), axis=0).reshape(1,-1)[0].tolist()

    
    return dict_points

def nsw(query_point: np.ndarray, all_documents: np.ndarray, 
        graph_edges: Dict[int, List[int]],
        search_k: int = 10, num_start_points: int = 5,
        dist_f: Callable = distance) -> np.ndarray:
    # допишите ваш код здесь 
    ii = 0
    jj = 0
    visited_points_dict = OrderedDict()
    seen_points_dict = OrderedDict()
    
    N_docs = np.shape(all_documents)[0]
    
    num_point_i = 0
    while (num_start_points > num_point_i) or (len(seen_points_dict) < search_k):
        jj += 1
        
#         new_start_point = np.random.choice(range(N_docs), 1)[0] ## EXTREMELY SLOW
        new_start_point = np.random.randint(0, N_docs)
        if new_start_point in visited_points_dict.keys():
#             num_point_i += 1
            continue
            
        if new_start_point in seen_points_dict.keys():
            cur_dist = seen_points_dict[new_start_point]
        else:
            cur_dist = dist_f(pointA=query_point,\
                documents=all_documents[[new_start_point], ])[0][0]
        
        visited_points_dict[new_start_point] = cur_dist
        is_new_point_added = False
        
        while not is_new_point_added:
            ii += 1
            
            new_edges_num = graph_edges[new_start_point]
            prev_new_start_point = new_start_point
            
            for edge_num in new_edges_num:
                if edge_num not in seen_points_dict.keys():
                    new_distance = dist_f(pointA=query_point,\
                        documents=all_documents[[edge_num], ])[0][0]
                    seen_points_dict[edge_num] = new_distance
                
                if cur_dist > seen_points_dict[edge_num]:
                    new_start_point = edge_num
                    cur_dist = seen_points_dict[edge_num]
            
            if prev_new_start_point == new_start_point:
                num_point_i += 1
                break
            
            if new_start_point in visited_points_dict.keys():
                num_point_i += 1
                break
            else:
                visited_points_dict[new_start_point] = cur_dist
    
    print(ii)
    print(jj)  
    top_k_points_i = np.argsort(list(seen_points_dict.values()))[:search_k]
    top_k_points_nums = np.array(list(seen_points_dict.keys()))[top_k_points_i]
    return top_k_points_nums


# funs draft

## distance

In [282]:
def distance(pointA: np.ndarray, documents: np.ndarray) -> np.ndarray:
    # допишите ваш код здесь 
    return np.sqrt(np.sum((documents - pointA) ** 2, axis=1)).reshape(-1,1)

In [80]:
np.ndarray((1, 6), dtype=float)

array([[4.67432274e-310, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000]])

In [81]:
np.ndarray((5, 6), dtype=float)

array([[0.00000000e+000, 4.67432231e-310, 1.02587682e-071,
        9.06324828e-043, 4.27255707e+180, 7.94651531e-061],
       [1.12547816e-153, 6.01347002e-154, 3.74162949e+233,
        1.35717430e+131, 9.42218944e-096, 5.88365854e-062],
       [4.61101863e+252, 4.55072025e+198, 2.87341591e+159,
        6.09946325e-154, 5.51675406e-048, 6.01347002e-154],
       [6.01353877e-154, 2.21294506e-052, 5.94084270e-038,
        7.61402148e-010, 4.34905137e+199, 1.67354582e+243],
       [1.05894728e-153, 8.96864382e+130, 1.72731109e+097,
        9.08967387e+242, 5.36812262e+242, 5.50409548e+257]])

In [82]:
# np.zeros((4, 5))
tst_arr1 = np.random.randint(low=5, size=(6,3))
tst_arr1

array([[3, 0, 3],
       [0, 1, 4],
       [3, 2, 1],
       [1, 2, 2],
       [4, 4, 4],
       [4, 4, 1]])

In [83]:
tst_arr0 = tst_arr1[0, ]
tst_arr0

array([3, 0, 3])

In [84]:
np.shape(tst_arr0)

(3,)

In [85]:
np.shape(tst_arr1)

(6, 3)

In [86]:
cur_sum = (tst_arr1 - tst_arr0) ** 2
cur_sum

array([[ 0,  0,  0],
       [ 9,  1,  1],
       [ 0,  4,  4],
       [ 4,  4,  1],
       [ 1, 16,  1],
       [ 1, 16,  4]])

In [87]:
np.sqrt(np.sum(cur_sum, axis=1)).reshape(-1,1)

array([[0.        ],
       [3.31662479],
       [2.82842712],
       [3.        ],
       [4.24264069],
       [4.58257569]])

In [88]:
np.linalg.norm(tst_arr1 - tst_arr0, axis=1, keepdims=True)

array([[0.        ],
       [3.31662479],
       [2.82842712],
       [3.        ],
       [4.24264069],
       [4.58257569]])

In [89]:
distance(tst_arr1, tst_arr0)

array([[0.        ],
       [3.31662479],
       [2.82842712],
       [3.        ],
       [4.24264069],
       [4.58257569]])

## create graph

In [43]:
def create_sw_graph(
        data: np.ndarray,
        num_candidates_for_choice_long: int = 10,
        num_edges_long: int = 5,
        num_candidates_for_choice_short: int = 10,
        num_edges_short: int = 5,
        use_sampling: bool = False,
        sampling_share: float = 0.05,
        dist_f: Callable = distance
    ) -> Dict[int, List[int]]:
    # допишите ваш код здесь 
    dict_points = dict()
    
    for i in tqdm(range(np.shape(data)[0])):
#     for i in range(np.shape(data)[0]):
        # random sampling for large datasets
        if use_sampling:
            N_points = int(sampling_share * np.shape(data)[0])
            N_points_idx = np.random.choice(np.shape(data)[0], N_points, replace=False)
            # distance from current point
            dist_i = distance(data[i, ], data[N_points_idx, ])
        else:
            dist_i = distance(data[i, ], data)
        
        # get indices of sorted values
        dist_i_argsort = np.argsort(dist_i, axis=0)
        
        # choose long edges (points)
        num_edges_long = np.min((num_candidates_for_choice_long, num_edges_long))
        long_rand_idx = np.random.choice(num_candidates_for_choice_long, num_edges_long,\
                                         replace=False)
        long_idx = dist_i_argsort[-num_candidates_for_choice_long:][long_rand_idx]
        
        
        # choose short edges (points)
        num_edges_short = np.min((num_candidates_for_choice_short, num_edges_short))
        short_rand_idx = np.random.choice(num_candidates_for_choice_short, num_edges_short,\
                                     replace=False)
        short_idx = dist_i_argsort[1:num_candidates_for_choice_short+1][short_rand_idx]
        
        # collect points' indices to dict
#         dict_points[i] = np.unique(np.concatenate(\
#             (long_idx, short_idx), axis=0).reshape(1,-1)[0]).tolist()
        dict_points[i] = np.concatenate(\
            (long_idx, short_idx), axis=0).reshape(1,-1)[0].tolist()

    
    return dict_points

In [320]:
tst_arr1 = np.random.randint(15, size=(11,3))
tst_arr0 = tst_arr1[0, ]
tst_arr1

array([[12, 14,  3],
       [10, 12,  3],
       [ 1,  2,  3],
       [ 8,  0,  7],
       [ 8,  1,  0],
       [ 4, 14,  1],
       [ 3,  4,  1],
       [ 2,  4, 14],
       [ 0,  2,  4],
       [ 4,  4, 14],
       [ 7, 13,  0]])

In [321]:
t_dist = distance(tst_arr1, tst_arr0)
t_dist

array([[ 0.        ],
       [ 2.82842712],
       [16.2788206 ],
       [15.09966887],
       [13.92838828],
       [ 8.24621125],
       [13.60147051],
       [17.91647287],
       [17.        ],
       [16.88194302],
       [ 5.91607978]])

In [322]:
np.argsort(t_dist, axis=0)[2:]
t_arg_max = np.argsort(t_dist, axis=0)[-4:] # long choice cand
t_arg_max

array([[2],
       [9],
       [8],
       [7]])

In [323]:
np.argsort(t_dist, axis=0)[:4] # short choice cand

array([[ 0],
       [ 1],
       [10],
       [ 5]])

In [324]:
np.argsort(t_dist, axis=0)[:4][[0,2]] # short choice cand

array([[ 0],
       [10]])

In [325]:
np.argsort(t_dist, axis=0)[1:4+1][[0,2]] # short choice cand

array([[1],
       [5]])

In [326]:
t_arg_min = np.argsort(t_dist, axis=0)[:4][np.array((0,2))] # short choice cand
t_arg_min

array([[ 0],
       [10]])

In [327]:
np.random.choice(3, 3, replace=False)

array([1, 0, 2])

In [328]:
np.random.choice(10, 3, replace=False).tolist()

[5, 7, 2]

In [329]:
t_arg_max.reshape(1,-1)[0]

array([2, 9, 8, 7])

In [330]:
np.concatenate((t_arg_min.reshape(1,-1)[0], t_arg_max.reshape(1,-1)[0]))

array([ 0, 10,  2,  9,  8,  7])

In [331]:
np.concatenate((t_arg_min, t_arg_max), axis=0).reshape(1,-1)[0]

array([ 0, 10,  2,  9,  8,  7])

In [332]:
create_sw_graph(data=tst_arr1)

{0: [7, 9, 1, 2, 3, 1, 8, 5, 9, 7],
 1: [7, 6, 8, 3, 5, 5, 8, 2, 9, 7],
 2: [3, 1, 9, 6, 7, 1, 6, 3, 10, 9],
 3: [0, 1, 8, 2, 9, 5, 10, 4, 1, 8],
 4: [9, 0, 10, 2, 1, 9, 7, 2, 1, 8],
 5: [3, 7, 6, 1, 8, 6, 0, 8, 9, 1],
 6: [2, 1, 5, 0, 9, 0, 7, 4, 2, 8],
 7: [9, 8, 1, 6, 4, 0, 2, 8, 1, 5],
 8: [4, 7, 0, 3, 6, 7, 1, 6, 10, 3],
 9: [5, 8, 6, 10, 0, 8, 4, 7, 5, 2],
 10: [2, 8, 7, 5, 1, 6, 0, 4, 9, 3]}

In [333]:
int(0.2 * 7)

1

## search nn

In [365]:
# def nsw(query_point: np.ndarray, all_documents: np.ndarray, 
#         graph_edges: Dict[int, List[int]],
#         search_k: int = 10, num_start_points: int = 5,
#         dist_f: Callable = distance) -> np.ndarray:
#     # допишите ваш код здесь 
#     choosen_points = []
#     visited_points = []
#     choosen_points_dist = []
#     prev_start_points = []
    
#     N_docs = np.shape(all_documents)[0]
# #     print('N_docs')
# #     print(N_docs)
    
#     num_point_i = 0
#     while (num_start_points > num_point_i) or (len(choosen_points) < search_k):
# #     for num_point_i in range(num_start_points):
# #     while (len(choosen_points) < search_k):
# #     while (len(graph_edges) > len(choosen_points)) and (len(choosen_points) < search_k * 10):
# #     for i in range(15):
# #         print('i ' + str(i))
        
#         # init random point & start distance
# #         new_start_point = np.random.choice(N_docs, 1)
#         new_start_point = np.random.choice(np.setdiff1d(range(N_docs), visited_points), 1)
# #         new_start_point = np.random.choice(np.setdiff1d(range(N_docs), choosen_points), 1)
    
# #         while new_start_point in choosen_points:
# #         while (new_start_point in visited_points):
# #             new_start_point = np.random.choice(N_docs, 1)
#         visited_points.append(int(new_start_point))
        
#         cur_dist = dist_f(pointA=query_point, documents=all_documents[new_start_point, ])[0]
#         is_new_point_added = False
        
#         while not is_new_point_added:

# #             print('cur_dist')
# #             print(cur_dist)
# #             print(new_start_point)
# #             print(graph_edges[int(new_start_point)])
# #             print(all_documents[graph_edges[int(new_start_point)], ])

#             # check for closer point with edge from start
# #             new_edges_num = np.setdiff1d(graph_edges[int(new_start_point)], choosen_points)
# #             new_edges_num = np.setdiff1d(new_edges_num, visited_points)
# #             new_edges_num = np.setdiff1d(graph_edges[int(new_start_point)], visited_points)
#             new_edges_num = graph_edges[int(new_start_point)]
            
# #             new_edges_num = np.setdiff1d(graph_edges[int(new_start_point)],\
# #                                          np.concatenate((choosen_points, visited_points)))
            
# #             new_distances = dist_f(pointA=all_documents[new_start_point, ],\
# #                                    documents=all_documents[new_edges_num, ])
# #             if len(new_edges_num) == 0:
# #                 break
# #                 continue
# #             if (len(new_edges_num) == 0):
# #                 num_point_i += 1
# #                 choosen_points.append(int(new_start_point))
# #                 choosen_points_dist.append(float(cur_dist))
# #                 is_new_point_added = True
# #                 visited_points.append(int(new_start_point))
# #                 print('empty edges array')
# #                 break
            
#             new_distances = dist_f(pointA=query_point,\
#                                    documents=all_documents[new_edges_num, ])
            
# #             print('new_distances')
# #             print(new_distances)
# #             print(np.min(new_distances))
# #             print(cur_dist)
# #             print(np.min(new_distances) > cur_dist)
            
#             # if there is closer distance check choosen points & update cur_dist
# #             try:
#             if (np.min(new_distances) < cur_dist) and (len(new_edges_num) > 0):
#                 cur_dist = np.min(new_distances)
#                 new_start_point = new_edges_num[np.argmin(new_distances)]
                
#                 if new_start_point in visited_points:
#                     num_point_i += 1
#                     break
#                 else:
#                     visited_points.append(int(new_start_point))
            
#             elif (len(new_edges_num) > 0):
#                 choosen_points.append(int(new_start_point))
#                 choosen_points_dist.append(float(cur_dist))
#                 visited_points.append(int(new_start_point))
#                 num_point_i += 1
#                 is_new_point_added = True
# #             else:
# #                 is_new_point_added = True
# #                 visited_points.append(int(new_start_point))
# #                 print('empty edges array')
# #             except:
# #                 print('new_distances')
# #                 print(new_distances)
# #                 print('cur_dist')
# #                 print(cur_dist)
# #                 print('new_edges_num')
# #                 print(new_edges_num)
                
                
# #         print('choosen_points ' + str(choosen_points))
    
#     return np.array(choosen_points)[np.argsort(choosen_points_dist)[:search_k]]


In [414]:
def nsw(query_point: np.ndarray, all_documents: np.ndarray, 
        graph_edges: Dict[int, List[int]],
        search_k: int = 10, num_start_points: int = 5,
        dist_f: Callable = distance) -> np.ndarray:
    # допишите ваш код здесь 
    ii = 0
    jj = 0
    visited_points_dict = OrderedDict()
    seen_points_dict = OrderedDict()
    
    N_docs = np.shape(all_documents)[0]
    
    num_point_i = 0
    while (num_start_points > num_point_i) or (len(seen_points_dict) < search_k):
        jj += 1
        
#         new_start_point = np.random.choice(range(N_docs), 1)[0] ## EXTREMELY SLOW
        new_start_point = np.random.randint(0, N_docs)
        if new_start_point in visited_points_dict.keys():
#             num_point_i += 1
            continue
            
        if new_start_point in seen_points_dict.keys():
            cur_dist = seen_points_dict[new_start_point]
        else:
            cur_dist = dist_f(pointA=query_point,\
                documents=all_documents[[new_start_point], ])[0][0]
        
        visited_points_dict[new_start_point] = cur_dist
        is_new_point_added = False
        
        while not is_new_point_added:
            ii += 1
            
            new_edges_num = graph_edges[new_start_point]
            prev_new_start_point = new_start_point
            
            for edge_num in new_edges_num:
                if edge_num not in seen_points_dict.keys():
                    new_distance = dist_f(pointA=query_point,\
                        documents=all_documents[[edge_num], ])[0][0]
                    seen_points_dict[edge_num] = new_distance
                
                if cur_dist > seen_points_dict[edge_num]:
                    new_start_point = edge_num
                    cur_dist = seen_points_dict[edge_num]
            
            if prev_new_start_point == new_start_point:
                num_point_i += 1
                break
            
            if new_start_point in visited_points_dict.keys():
                num_point_i += 1
                break
            else:
                visited_points_dict[new_start_point] = cur_dist
    
    print(ii)
    print(jj)  
    top_k_points_i = np.argsort(list(seen_points_dict.values()))[:search_k]
    top_k_points_nums = np.array(list(seen_points_dict.keys()))[top_k_points_i]
    return top_k_points_nums


In [390]:
%load_ext line_profiler

In [391]:
%lprun -f nsw nsw(query_point=np.array((1,2,3)), all_documents=tst_arr1, graph_edges=t_sw_gr,\
    num_start_points=1000,\
    search_k=50)

1954
1000


In [417]:
%%time
nsw(query_point=np.array((1,2,3)), all_documents=tst_arr1, graph_edges=t_sw_gr,\
    num_start_points=1000,\
    search_k=50)

2132
1090
CPU times: user 170 ms, sys: 362 µs, total: 170 ms
Wall time: 165 ms


array([ 7353,  9193,  7017, 13964,  2079, 13232,  4372, 11377, 14128,
        3788, 10516, 10572, 12669, 11803, 14740,  4284,  8298,  1696,
        8271,  2371,  8997,  8941,  5741,  8697,  6548,  3350,  2668,
       12062, 11537,  1925,  6427, 10817,  4989, 13639,  6106, 11894,
        3844,  5634,  4466,  6959, 14288,  1633,  8660, 10760,  8089,
        1722, 10378, 12567, 12347,  7258])

In [362]:
%%time
nsw(query_point=np.array((1,2,3)), all_documents=tst_arr1, graph_edges=t_sw_gr,\
    num_start_points=1000,\
    search_k=50)

2355
1228
CPU times: user 200 ms, sys: 22 ms, total: 222 ms
Wall time: 197 ms


array([ 2371, 11377,  3788,  8941,  6548,  8298,  8271, 12669, 10572,
       14740, 10516, 11537,  5587, 14020, 10378,  6116,  4989, 12567,
        1633, 11126, 10760,  4123,  7170, 11894, 13639, 10536, 10817,
        7258, 12347,  6546, 14650, 14288,  8092,  5634,  6106,  7504,
       14718,  1930, 12309,  5045, 13642,  5887,  5694,  3341, 11607,
        9759,  5944,  1137,  2608,  2973])

In [4]:
(1,2) in [1,2,3]

False

In [47]:
%%time
tst_arr1 = np.random.randint(15, size=(15000,3))
tst_arr0 = tst_arr1[0, ]
tst_arr1

CPU times: user 1.36 ms, sys: 0 ns, total: 1.36 ms
Wall time: 782 µs


array([[14, 10, 14],
       [ 9, 12, 12],
       [ 8,  6,  1],
       ...,
       [ 5, 14,  0],
       [10,  7,  3],
       [ 0, 11, 13]])

In [48]:
%%time
t_sw_gr = create_sw_graph(data=tst_arr1)
t_sw_gr[int(np.array(0))]


CPU times: user 26.6 s, sys: 205 ms, total: 26.8 s
Wall time: 26.5 s


[5435, 2277, 3608, 10184, 7563, 1457, 10496, 10956, 14900, 11502]

In [377]:
t_sw_gr

{0: [5435, 2277, 3608, 10184, 7563, 1457, 10496, 10956, 14900, 11502],
 1: [13304, 10184, 5435, 2943, 9878, 1, 1136, 7774, 10173, 14072],
 2: [13503, 14731, 1847, 8075, 4637, 2, 7194, 12473, 10916, 11705],
 3: [387, 4968, 12933, 14811, 3783, 14923, 3, 12457, 14684, 14096],
 4: [569, 12095, 6745, 4518, 11229, 6094, 1507, 10951, 10833, 7717],
 5: [13503, 5620, 8075, 7440, 14300, 5, 9360, 11496, 14895, 14338],
 6: [3619, 12591, 5118, 7620, 7015, 13137, 6931, 12364, 12481, 13337],
 7: [9633, 12874, 6745, 1464, 3416, 394, 11555, 8048, 3061, 6429],
 8: [8016, 1557, 10893, 2365, 2546, 12387, 5460, 4359, 14826, 2824],
 9: [7506, 12834, 12399, 6823, 1208, 3092, 11782, 6682, 9927, 2576],
 10: [3862, 1721, 4968, 3783, 13279, 9421, 6690, 2173, 10, 9969],
 11: [3242, 3274, 10893, 2137, 5507, 774, 1867, 6335, 13501, 12466],
 12: [3353, 12874, 9940, 14824, 1464, 1677, 9608, 10050, 8112, 11898],
 13: [13304, 4153, 5435, 1953, 14962, 10796, 8342, 3396, 3877, 4996],
 14: [1953, 7975, 3608, 3132, 4153, 1

In [343]:
np.Inf < 1

False

In [407]:
np.setdiff1d([1,2,3], [2,3,4])

array([1])

In [418]:
np.min(np.array((4,1,2,3)).reshape(-1,1))

1

In [444]:
if (np.min(np.array((1,2,3)).reshape(-1,1)) < np.array((4))) and ([1,2]):
    print('ok')

ok


In [73]:
np.setdiff1d(range(5), (1,2))

array([0, 3, 4])

In [86]:
np.random.choice(np.setdiff1d(range(5), (1,2)), 1)

array([4])

In [108]:
# np.min(new_distances)


In [140]:
t_aa = []
t_dd = dict()

In [138]:
%timeit t_aa.append(1)

84.8 ns ± 6.73 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [141]:
%timeit t_dd[1] = True

62.7 ns ± 3.98 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [148]:
t_ar = np.array(range(1000)).tolist()

In [149]:
%timeit 100 in t_ar

1.51 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [161]:
dfd0 = defaultdict()
dfd0[1] = True

In [162]:
dfd0

defaultdict(None, {1: True})

In [178]:
dfd0.keys()

dict_keys([1])

In [165]:
1 in dfd0.keys()

True

In [166]:
1 in dfd0

True

In [167]:
2 in dfd0

False

In [168]:
np.setdiff1d(range(3), dfd0)

array([0, 1, 2])

In [181]:
%timeit np.setdiff1d(range(3), list(dfd0.keys()))

44.7 µs ± 4.99 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [182]:
%timeit np.setdiff1d(range(3), dfd0.keys())

48.3 µs ± 4.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [183]:
%timeit np.setdiff1d(range(3), 1)

40.9 µs ± 4.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [172]:
np.setdiff1d(range(3), [1])

array([0, 2])

In [194]:
for i in range(10):
    if True:
        print(i)
        if True:
            break

0


In [294]:
for i in range(10):
    if True:
        print(i)
else:
    print(i + 100)

0
1
2
3
4
5
6
7
8
9
109


In [305]:
dfd0[2] = 5

In [306]:
len(dfd0)

2

In [314]:
# np.sort(dfd0)
np.argsort(list(dfd0.values()))

array([0, 1])

In [419]:
%timeit np.random.choice(range(100), 1)

41.8 µs ± 3.76 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [421]:
%timeit np.random.choice(range(1000), 1)

156 µs ± 7.02 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [423]:
%timeit np.random.choice(range(10000), 1)

1.38 ms ± 81.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [420]:
%timeit np.random.randint(0, 100)

3.69 µs ± 348 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [422]:
%timeit np.random.randint(0, 1000)

3.84 µs ± 339 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
